In [108]:
import pandas as pd
import glob
import json
from collections import defaultdict

In [109]:
data = pd.read_csv("chicago_phoenix_utf8.csv", encoding="utf-8")
data = data.sort_values(['postal_code'], ignore_index=True)

In [110]:
data.head()

,business_id,business_name,business_url,location,is_business_closed,year_established,overall_rating,num_reviews,menu_url,menu_items_scraped_flag,...,num_reviews_2_stars,num_reviews_1_star,address_line1,city,region_code,postal_code,country_code,categories,top_food_items,monthly_ratings_by_year
0,2aVQu4wnptrFM_kxDhhPIg,Freshly,/biz/freshly-phoenix-2,"Phoenix, AZ",0,NaN,3.0,835,NaN,0,...,102,236,NaN,New York,NY,10010,US,"{Food Delivery Services,Gluten-Free}","{buffalo chicken,peppercorn steak,chicken parm...","{""2016"": [[0, 4.5], [1, 4.5], [2, 4], [3, 4], ..."
1,w91a9VtjbySkCAT0jiH9-w,Des Plaines Family Restaurant & Pancake House,/biz/des-plaines-family-restaurant-and-pancake...,"Chicago, IL",0,NaN,4.5,196,NaN,0,...,4,9,1769 Miner St,Des Plaines,IL,60016,US,{Breakfast & Brunch},"{french toast,santa fe omelettte,veggie skillet}","{""2016"": [[0, 4.5], [1, 4], [2, 4.5], [3, 5], ..."
2,liJHw9dOpjCQlW6nzr01YQ,Hofbrauhaus Chicago,/biz/hofbrauhaus-chicago-rosemont-4,"Chicago, IL",0,2013.0,3.0,972,/menu/hofbrauhaus-chicago-rosemont-4/lunch-menu,0,...,157,184,5500 Park Pl,Rosemont,IL,60018,US,"{German,Breweries}","{pork shank,jagerschnitzel,sausage plate}","{""2016"": [[0, 2], [1, 3.5], [2, 3], [3, 4], [4..."
3,IDs2ziQg69KufhDXtmyIRQ,Park Tavern,/biz/park-tavern-rosemont,"Chicago, IL",0,2012.0,3.0,668,NaN,0,...,96,157,5433 Park Pl,Rosemont,IL,60018,US,"{Gastropubs,American (New)}","{fish tacos,foot long mac,bacon mac,chopped sa...","{""2016"": [[0, 3.5], [1, 3], [2, 2.5], [3, 3], ..."
4,PyQlb3qediXEbQdQE1ryew,Mariachi’s Tacos,/biz/mariachis-tacos-des-plaines,"Chicago, IL",0,NaN,4.5,200,NaN,0,...,6,11,1936 E Touhy Ave,Des Plaines,IL,60018,US,{Mexican},"{al pastor tacos,carne asada,shrimp tacos,mari...","{""2016"": [[6, 5], [7, 4.5], [8, 4.5], [9, 4.5]..."


In [111]:
df = data[["business_id", "business_name", "location", "num_reviews", "postal_code"]]

In [112]:
df.head()

,business_id,business_name,location,num_reviews,postal_code
0,2aVQu4wnptrFM_kxDhhPIg,Freshly,"Phoenix, AZ",835,10010
1,w91a9VtjbySkCAT0jiH9-w,Des Plaines Family Restaurant & Pancake House,"Chicago, IL",196,60016
2,liJHw9dOpjCQlW6nzr01YQ,Hofbrauhaus Chicago,"Chicago, IL",972,60018
3,IDs2ziQg69KufhDXtmyIRQ,Park Tavern,"Chicago, IL",668,60018
4,PyQlb3qediXEbQdQE1ryew,Mariachi’s Tacos,"Chicago, IL",200,60018


In [113]:
df.shape

(2902, 5)

In [114]:
filenames = glob.glob("./reviews with detected entities/*/*")

In [115]:
rating = []
business_name = []
business_id = []
business_location = []
menu_list = []
count_list = []
# entities = defaultdict(int)

for file in filenames:
    entities = defaultdict(int)

    with open(file, "r") as line:
        json_list = list(line)
    
    i = 0
    rate = 0
    num_reviews_ = 0

    for review in json_list:
        reviewJson = json.loads(review)

        if i == 0:
            business_name.append(reviewJson['business_name'])
            business_id.append(reviewJson['business_id'])
            business_location.append(reviewJson['business_location'])
            i += 1
        
        rate += reviewJson['rating']
        num_reviews_ += 1

        for item in reviewJson['entities']:
            entities[item] += 1

    rating.append(rate / num_reviews_)
    sorted_menu = sorted(entities.items(), key=lambda kv: kv[1], reverse=True)
    menu = str([i for i, j in sorted_menu][:100]).replace("\'", '"')
    count = str([j for i, j in sorted_menu][:100])

    menu_list.append(menu)
    count_list.append(count)
    

In [116]:
print(len(rating))
print(len(business_name))
print(len(business_id))
print(len(business_location))
print(len(menu_list))
print(len(count_list))

1999
1999
1999
1999
1999
1999


In [117]:
bus_with_menus = pd.DataFrame({"business_id":business_id,
                                    "business_name": business_name,
                                    "business_location": business_location,
                                    "rating": rating,
                                    "menu_list": menu_list,
                                    "count":count_list})

In [118]:
bus_with_menus.head()

,business_id,business_name,business_location,rating,menu_list,count
0,ksY97mbEy23n8GbYxOBgpA,25 Degrees,"Chicago, IL",4.059016,"[""burger"", ""burgers"", ""fries"", ""sweet potato f...","[428, 406, 224, 153, 134, 124, 106, 93, 86, 83..."
1,_wtMT97gelNAyfI82FPFvA,3 Arts Club Cafe,"Chicago, IL",4.331250,"[""chicago"", ""meal"", ""fountain"", ""fries"", ""lobs...","[60, 38, 37, 36, 31, 26, 24, 22, 22, 21, 20, 1..."
2,JaO7VswL5hk1DF09CYB-cQ,312 Chicago,"Chicago, IL",3.674833,"[""meal"", ""chicago"", ""dessert"", ""bread"", ""itali...","[87, 68, 65, 45, 41, 39, 35, 31, 30, 30, 26, 2..."
3,7mCeefskNUtOkzxt_MLHKA,3rd Coast Cafe,"Chicago, IL",4.220949,"[""breakfast"", ""chicago"", ""meal"", ""hash browns""...","[214, 82, 58, 52, 40, 35, 34, 31, 28, 27, 24, ..."
4,0pc352jFnUQFAw8DgmeSjw,5 Rabanitos Restaurante and Taqueria,"Chicago, IL",4.146237,"[""mexican"", ""tacos"", ""byob"", ""authentic"", ""gua...","[131, 89, 87, 61, 60, 54, 50, 45, 43, 39, 39, ..."


In [120]:
df.loc[:,"postal_code"] = df["postal_code"].astype("str")

C:\Users\bhavi\miniconda3\envs\bda-hw\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [121]:
temp = bus_with_menus.merge(df[["business_id", "postal_code", "num_reviews"]], on="business_id", how="left")

In [122]:
temp.head()

,business_id,business_name,business_location,rating,menu_list,count,postal_code,num_reviews
0,ksY97mbEy23n8GbYxOBgpA,25 Degrees,"Chicago, IL",4.059016,"[""burger"", ""burgers"", ""fries"", ""sweet potato f...","[428, 406, 224, 153, 134, 124, 106, 93, 86, 83...",60654,918.0
1,_wtMT97gelNAyfI82FPFvA,3 Arts Club Cafe,"Chicago, IL",4.331250,"[""chicago"", ""meal"", ""fountain"", ""fries"", ""lobs...","[60, 38, 37, 36, 31, 26, 24, 22, 22, 21, 20, 1...",60610,1094.0
2,JaO7VswL5hk1DF09CYB-cQ,312 Chicago,"Chicago, IL",3.674833,"[""meal"", ""chicago"", ""dessert"", ""bread"", ""itali...","[87, 68, 65, 45, 41, 39, 35, 31, 30, 30, 26, 2...",60602,447.0
3,7mCeefskNUtOkzxt_MLHKA,3rd Coast Cafe,"Chicago, IL",4.220949,"[""breakfast"", ""chicago"", ""meal"", ""hash browns""...","[214, 82, 58, 52, 40, 35, 34, 31, 28, 27, 24, ...",60610,627.0
4,0pc352jFnUQFAw8DgmeSjw,5 Rabanitos Restaurante and Taqueria,"Chicago, IL",4.146237,"[""mexican"", ""tacos"", ""byob"", ""authentic"", ""gua...","[131, 89, 87, 61, 60, 54, 50, 45, 43, 39, 39, ...",60608,480.0


In [123]:
temp = temp.rename({"business_name":"name", "business_location":"city", "postal_code":"zipcode", "menu_list":"menu"}, axis=1)

In [127]:
temp[["business_id", "name", "city", "zipcode", "rating", "num_reviews", "menu", "count"]].head()

,business_id,name,city,zipcode,rating,num_reviews,menu,count
0,ksY97mbEy23n8GbYxOBgpA,25 Degrees,"Chicago, IL",60654,4.059016,918.0,"[""burger"", ""burgers"", ""fries"", ""sweet potato f...","[428, 406, 224, 153, 134, 124, 106, 93, 86, 83..."
1,_wtMT97gelNAyfI82FPFvA,3 Arts Club Cafe,"Chicago, IL",60610,4.331250,1094.0,"[""chicago"", ""meal"", ""fountain"", ""fries"", ""lobs...","[60, 38, 37, 36, 31, 26, 24, 22, 22, 21, 20, 1..."
2,JaO7VswL5hk1DF09CYB-cQ,312 Chicago,"Chicago, IL",60602,3.674833,447.0,"[""meal"", ""chicago"", ""dessert"", ""bread"", ""itali...","[87, 68, 65, 45, 41, 39, 35, 31, 30, 30, 26, 2..."
3,7mCeefskNUtOkzxt_MLHKA,3rd Coast Cafe,"Chicago, IL",60610,4.220949,627.0,"[""breakfast"", ""chicago"", ""meal"", ""hash browns""...","[214, 82, 58, 52, 40, 35, 34, 31, 28, 27, 24, ..."
4,0pc352jFnUQFAw8DgmeSjw,5 Rabanitos Restaurante and Taqueria,"Chicago, IL",60608,4.146237,480.0,"[""mexican"", ""tacos"", ""byob"", ""authentic"", ""gua...","[131, 89, 87, 61, 60, 54, 50, 45, 43, 39, 39, ..."


In [128]:
temp[["business_id", "name", "city", "zipcode", "rating", "num_reviews", "menu", "count"]].to_csv("final_data_v3.csv")